<a href="https://colab.research.google.com/github/Adithya6ramesh/LangChain/blob/main/PDFQuery_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart: Querying PDF With Astra and LangChain

### A question-answering demo using Astra DB and LangChain, powered by Vector Search

#### Pre-requisites:

You need a **_Serverless Cassandra with Vector Search_** database on [Astra DB](https://astra.datastax.com) to run this demo. As outlined in more detail [here](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html#_prepare_for_using_your_vector_database), you should get a DB Token with role _Database Administrator_ and copy your Database ID: these connection parameters are needed momentarily.

You also need an [OpenAI API Key](https://cassio.org/start_here/#llm-access) for this demo to work.

#### What you will do:

- Setup: import dependencies, provide secrets, create the LangChain vector store;
- Run a Question-Answering loop retrieving the relevant headlines and having an LLM construct the answer.

Install the required dependencies:

In [8]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Import the packages you'll need:

In [10]:
!pip install -U langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.3 MB/s eta 0:00:00


In [11]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [12]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00


In [15]:
from PyPDF2 import PdfReader

### Setup

In [17]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:FSqbIBGYatjDMrtzjbwuomiD:bca79fa5d43aaccd39e9c9e7eebca7e83469c3944149ae38fe7bf6348df30068" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "ade7bc0b-d5ac-479b-aaed-79c1a6aa2630" # enter your Database ID

OPENAI_API_KEY = "sk-J3ZbnEqytFesD7kWKuVaT3BlbkFJl7Vr9dpDbViv2R2un" # enter your OpenAI key

#### Provide your secrets:

Replace the following with your Astra DB connection details and your OpenAI API key:

In [18]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/Expt No.7.pdf')

In [19]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [20]:
raw_text

'RING  & JOHNSON  COUNTER USING  FLIPFLOPS  \n \nAim:  To design  and setup  4 bit ring counter  and Johnson  counter  using  flip- flops  \nComponents&  Equipments  Required:  \n \nIC 7474,  IC 7476 - 2 Nos. &IC trainer  kit \n \nTheory:  \n \nShift counters give a specific set of counter states. It is obtained by shifting a specific  \narray  of binary  inputs. Ring  counter and Johnson counter  are two important shift counters. A \nregister is simply a group of flipflop that can be used to store a binary number or a group of  \nflipflops connected to provided either or both of these functions is called a shift register. shift  \nregister is a type of digital circuit using a cascade of flip flops where the output of one flip -flop is  \nconnected to the input of the next. They share a single clock pulse, which causes the data stored  \nin the system to shift from one location to the next. There are two ways to shift data into the  \nregister ( serial  and parallel) and similarly two 

Initialize the connection to your database:

_(do not worry if you see a few warnings, it's just that the drivers are chatty about negotiating protocol versions with the DB.)_

In [21]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(139238380282080) ade7bc0b-d5ac-479b-aaed-79c1a6aa2630-us-east1.db.astra.datastax.com:29042:13211a36-5dc7-4c6c-989e-0df5fa03f1bc> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Create the LangChain embedding and LLM objects for later usage:

In [22]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Create your LangChain vector store ... backed by Astra DB!

In [23]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-J3Zbn************************************R2un. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [24]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [25]:
texts[:50]

['RING  & JOHNSON  COUNTER USING  FLIPFLOPS  \n \nAim:  To design  and setup  4 bit ring counter  and Johnson  counter  using  flip- flops  \nComponents&  Equipments  Required:  \n \nIC 7474,  IC 7476 - 2 Nos. &IC trainer  kit \n \nTheory:  \n \nShift counters give a specific set of counter states. It is obtained by shifting a specific  \narray  of binary  inputs. Ring  counter and Johnson counter  are two important shift counters. A \nregister is simply a group of flipflop that can be used to store a binary number or a group of  \nflipflops connected to provided either or both of these functions is called a shift register. shift  \nregister is a type of digital circuit using a cascade of flip flops where the output of one flip -flop is',
 'register is a type of digital circuit using a cascade of flip flops where the output of one flip -flop is  \nconnected to the input of the next. They share a single clock pulse, which causes the data stored  \nin the system to shift from one locatio

### Load the dataset into the vector store



In [26]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

NameError: name 'astra_vector_store' is not defined

### Run the QA cycle

Simply run the cells and ask a question -- or `quit` to stop. (you can also stop execution with the "▪" button on the top toolbar)

Here are some suggested questions:
- _What is the current GDP?_
- _How much the agriculture target will be increased to and what the focus will be_


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): How much the agriculture target will be increased to and what the focus will be

QUESTION: "How much the agriculture target will be increased to and what the focus will be"


ANSWER: "The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."

What's your next question (or type 'quit' to exit): What is the current GDP

QUESTION: "What is the current GDP"


ANSWER: "The current GDP is estimated to be 7 per cent."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."

What's your next question (or type 'quit' to exit): quit
